In [1]:
import requests
from bs4 import BeautifulSoup
import json
from pprint import pprint
import numpy as np
import pandas as pd

In [2]:
res = requests.get('http://openapi.seoul.go.kr:8088/5058644a58646c733132324e64486957/json/bikeList/1/1000/')
data = json.loads(res.content)
df_1 = pd.DataFrame.from_dict(data['rentBikeStatus']['row'])


res_2 = requests.get('http://openapi.seoul.go.kr:8088/5058644a58646c733132324e64486957/json/bikeList/1001/2000')
data_2 = json.loads(res_2.content)
df_2 = pd.DataFrame.from_dict(data_2['rentBikeStatus']['row'])


fin_data = pd.concat([df_1, df_2],ignore_index=True)

In [3]:
df_use = pd.read_csv('using_No.csv', engine='python', encoding='utf-8-sig', names=['station_No'])

In [4]:
colnames=['자전거번호', '대여일시', '대여대여소번호', '대여대여소명', '거치대순번', '반납일시', '반납대여소번호',
       '반납대여소명', '반납거치대순번', '이용시간', '이용거리']

In [5]:
df_01   = pd.read_csv("서울특별시 공공자전거 대여정보_201901.csv", engine='python')       # 2019_1_1 ~ 2018_1_31
df_02   = pd.read_csv("서울특별시 공공자전거 대여정보_201902.csv", engine='python')       # 2019_2_1 ~ 2018_2_28
df_03   = pd.read_csv("서울특별시 공공자전거 대여정보_201903.csv", engine='python')       # 2019_3_1 ~ 2018_3_31
df_04   = pd.read_csv("서울특별시 공공자전거 대여정보_201904.csv", names=colnames, engine='python')       # 2019_4_1 ~ 2019_4_30
df_05   = pd.read_csv("서울특별시 공공자전거 대여정보_201905.csv", names=colnames, engine='python')       # 2019_5_1 ~ 2019_5_31

In [6]:
df_123 = pd.concat([df_01, df_02, df_03])

In [7]:
df_123 = df_123.drop(df_123.columns[[0,3,4,7,8,9,10]], axis=1)
df_123.rename(columns={'대여소번호':'대여대여소번호', '대여소번호.1':'반납대여소번호'}, inplace=True)

In [8]:
# df_123['반납대여소명'] = df_123['반납대여소명']+'.'
# df_123['대여대여소명'] = df_123['대여대여소명']+'.'
# df_123['반납대여소명'] = df_123['반납대여소명'].apply(lambda e: e.split('.')[1])
# df_123['대여대여소명'] = df_123['대여대여소명'].apply(lambda e: e.split('.')[1])

In [9]:
df_45 = pd.concat([df_04, df_05])
df_45 = df_45.drop(df_45.columns[[0,3,4,7,8,9,10]], axis=1)

In [10]:
df = pd.concat([df_123, df_45])
df.head()

,대여일시,대여대여소번호,반납일시,반납대여소번호
0,2019-01-01 00:02:16,1408,2019-01-01 00:07:07,1433
1,2019-01-01 00:03:09,615,2019-01-01 00:07:27,612
2,2019-01-01 00:01:44,1177,2019-01-01 00:08:09,1152
3,2019-01-01 00:02:28,646,2019-01-01 00:08:30,648
4,2019-01-01 00:04:14,2273,2019-01-01 00:08:54,2255


In [11]:
fin_data['station_No']=fin_data['stationName'].apply(lambda e: e.split('.')[0])
fin_data['station_Name']=fin_data['stationName'].apply(lambda e: e.split('.')[1])
fin_data = fin_data.drop(columns=['stationName'])
fin_data[:3]

,parkingBikeTotCnt,rackTotCnt,shared,stationId,stationLatitude,stationLongitude,station_No,station_Name
0,0,5,0,ST-3,37.54956055,126.90575409,101,(구)합정동 주민센터
1,10,20,50,ST-4,37.55564880,126.91062927,102,망원역 1번출구 앞
2,10,14,71,ST-5,37.55495071,126.91083527,103,망원역 2번출구 앞


In [12]:
arr = fin_data['station_No']

none_data =[]

for strarr in arr:
    if (df[df['대여대여소번호']==int(strarr)].shape[0]==0):
        none_data.append(strarr)
        
fin_data1 = fin_data[~fin_data["station_No"].isin(none_data)]
fin_data1 = fin_data1.reset_index(drop=True)

In [13]:
arr = fin_data['station_No']

none_data =[]

for strarr in arr:
    if (df[df['반납대여소번호']==int(strarr)].shape[0]==0):
        none_data.append(strarr)
        
fin_data2 = fin_data[~fin_data["station_No"].isin(none_data)]
fin_data2 = fin_data2.reset_index(drop=True)

In [14]:
fin_slice1 = fin_data1[['stationLatitude', 'stationLongitude', 'station_No']]
fin_slice1.rename(columns={'station_No': '대여대여소번호', 'stationLatitude':'대여위도','stationLongitude':'대여경도'}, inplace=True)

C:\Users\LEEJUNGEUN\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [15]:
df['대여대여소번호'] = df['대여대여소번호'].astype(str)
df['반납대여소번호'] = df['반납대여소번호'].astype(str)

In [17]:
df_merge = pd.merge(fin_slice1, df)

In [18]:
df_merge.head()

,대여위도,대여경도,대여대여소번호,대여일시,반납일시,반납대여소번호
0,37.54956055,126.90575409,101,2019-01-01 08:29:11,2019-01-01 08:34:57,151
1,37.54956055,126.90575409,101,2019-01-01 14:11:50,2019-01-01 14:17:24,106
2,37.54956055,126.90575409,101,2019-01-02 06:54:05,2019-01-02 07:02:12,152
3,37.54956055,126.90575409,101,2019-01-02 07:23:10,2019-01-02 07:30:27,104
4,37.54956055,126.90575409,101,2019-01-02 08:50:51,2019-01-02 08:57:21,105


In [20]:
fin_slice2 = fin_data2[['stationLatitude', 'stationLongitude', 'station_No']]
fin_slice2.rename(columns={'station_No': '반납대여소번호','stationLatitude':'반납위도','stationLongitude':'반납경도'}, inplace=True)

In [21]:
df_merge = pd.merge(fin_slice2, df_merge)

In [22]:
df_merge.head()

,반납위도,반납경도,반납대여소번호,대여위도,대여경도,대여대여소번호,대여일시,반납일시
0,37.54956055,126.90575409,101,37.54956055,126.90575409,101,2019-01-03 09:57:31,2019-01-03 10:11:35
1,37.54956055,126.90575409,101,37.54956055,126.90575409,101,2019-01-04 14:00:53,2019-01-04 14:06:32
2,37.54956055,126.90575409,101,37.54956055,126.90575409,101,2019-01-04 18:29:43,2019-01-04 19:15:47
3,37.54956055,126.90575409,101,37.54956055,126.90575409,101,2019-01-06 14:12:34,2019-01-06 14:59:50
4,37.54956055,126.90575409,101,37.54956055,126.90575409,101,2019-01-07 19:35:34,2019-01-07 20:08:06


In [28]:
df_merge.to_csv('df_bicycle.csv',encoding='utf-8-sig',index=False)